In [3]:
#https://www.oreilly.com/learning/how-do-i-compare-document-similarity-using-python

import gensim
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')
#print(dir(gensim))
import pyspark
import pandas as pd
import math
try: 
    type(sc)
except NameError:
    sc = pyspark.SparkContext('local[*]')    
    
type(sc)
import datetime
from pyspark.sql.functions import year, month, dayofmonth
from pyspark.sql.functions import lit
from pyspark.sql.functions import unix_timestamp
from html.parser import HTMLParser


import numpy as np

[nltk_data] Downloading package punkt to /home/mpicco/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
#sqlContext.read.csv('./datos_navent_fiuba/fiuba_2_postulantes_genero_y_edad.csv').option("header",True)
# cargo avisos
df_avisos1 = pd.read_csv('../datos_navent_fiuba/datos_navent/fiuba_6_avisos_detalle.csv')
df_avisos2 = pd.read_csv('../datos_navent_fiuba/fiuba_hasta_15_abril/fiuba_6_avisos_detalle.csv')
df_avisos3 = pd.read_csv('../datos_navent_fiuba/fiuba_desde_15_abril/fiuba_6_avisos_detalle.csv')
df_avisos4 = pd.read_csv('../datos_navent_fiuba/fiuba_desde_15_abril/fiuba_6_avisos_detalle_missing_nivel_laboral.csv')

df_avisos = df_avisos1.append(df_avisos2).append(df_avisos3).append(df_avisos4)

del df_avisos1
del df_avisos2
del df_avisos3
del df_avisos4

pd_dataframe= df_avisos.drop_duplicates(['idaviso'], keep='first')

print(df_avisos.shape)
print(df_avisos.head())
            
 

#reduzco las clomunas del df 
df_reduced= pd_dataframe.filter(["idaviso","descripcion"])
#filtro los campos que 
df_reduced= df_reduced[df_reduced["descripcion"].notnull()]

del pd_dataframe

#clase que parsea el texto
class MyHTMLParser(HTMLParser):
    
    def __init__(self):
        HTMLParser.__init__(self)
        self.Myrawdata = ""

    def handle_data(self, data):
        #self.Myrawdata = self.Myrawdata + data.split()
        self.Myrawdata = self.Myrawdata +" "+ data


def html_parser(texto):
    parser_aux = MyHTMLParser()
    parser_aux.feed(texto)
    return (parser_aux.Myrawdata)

#creo columna con el texto sin tags 
df_reduced["descripcion_limpia"] = df_reduced["descripcion"].apply(lambda x: html_parser(x))
df_reduced.drop("descripcion",1,inplace=True)
df_reduced= df_reduced.reset_index()


(45969, 11)
      idaviso  idpais                                         titulo  \
0     8725750       1               VENDEDOR/A PROVINCIA DE SANTA FE   
1    17903700       1                                     Enfermeras   
2  1000150677       1                                 Chofer de taxi   
3  1000610287       1  CHOFER DE CAMIONETA BAHIA BLANCA - PUNTA ALTA   
4  1000872556       1  Operarios de Planta - Rubro Electrodomésticos   

                                         descripcion        nombre_zona  \
0  <p><strong><strong>Empresa:</strong></strong> ...  Gran Buenos Aires   
1  <p>Solicitamos para importante cadena de farma...  Gran Buenos Aires   
2  <p>TE GUSTA MANEJAR? QUERES GANAR PLATA HACIEN...    Capital Federal   
3  <p><strong>Somos una empresa multinacional que...  Gran Buenos Aires   
4  <p><strong>OPERARIOS DE PLANTA</strong></p><p>...  Gran Buenos Aires   

  ciudad       mapacalle tipo_de_trabajo         nivel_laboral nombre_area  \
0    NaN             NaN  

In [5]:
lista_descripciones = [] 
for i in range(0,len(df_reduced["descripcion_limpia"])):
    lista_descripciones+=[df_reduced["descripcion_limpia"][i]]
    


In [6]:
raw_documents = lista_descripciones

In [7]:
from nltk.tokenize import word_tokenize
gen_docs = [[w.lower() for w in word_tokenize(text)] 
            for text in raw_documents]

In [8]:
dictionary = gensim.corpora.Dictionary(gen_docs)
print("Number of words in dictionary:",len(dictionary))
for i in range(len(dictionary)):
    print(i, dictionary[i])

Number of words in dictionary: 54281
0 (
1 )
2 ,
3 .
4 :
5 agrícolas
6 apoyo
7 argentina
8 automotor
9 automotriz
10 buena
11 buscamos
12 capacitación
13 cartera
14 ciudad
15 clientes
16 comisiones
17 comisión
18 completo
19 conocimiento
20 correas
21 de
22 dedicación
23 del
24 distribuidor
25 distribuidora
26 distribuímos
27 domicilio
28 e
29 empresa
30 en
31 es
32 excluyente
33 experiencia
34 fe
35 fuerte
36 full
37 full-time
38 gran
39 importador
40 importamos
41 indistinto
42 industria
43 industriales
44 influencia
45 la
46 logístico
47 mayorista
48 movilidad
49 nuestra
50 ofrecemos
51 para
52 país/ciudad
53 postulante
54 presencia
55 propia
56 provincia
57 puesto
58 ramo
59 representante
60 repuestos
61 requisitos
62 rosari
63 rosario
64 salario
65 santa
66 secundario
67 sexo
68 sobre
69 time
70 tipo
71 trabajo
72 un
73 vacantes:1
74 ventas
75 y
76 zona
77 área
78 23
79 3
80 45
81 a
82 atención
83 años
84 cadena
85 con
86 disponibilidad
87 enfermeras
88 farmacias
89 femenino
90 ha

In [9]:
corpus = [dictionary.doc2bow(gen_doc) for gen_doc in gen_docs]

In [10]:
postulacion_prueba = "Baker Tilly Argentina selecciona Asistentes para las áreas de Auditoria, Consultoría, Impuestos y Outsourcing. La búsqueda está orientada a recién graduados y/o estudiantes avanzados en las carreras de Contador, Lic. en Administración y/o Sistemas. Se ofrece desarrollo profesional y amplia capacitación en un excelente clima laboral"

In [11]:
tf_idf = gensim.models.TfidfModel(corpus)
print(tf_idf)

TfidfModel(num_docs=25288, num_nnz=2475320)


In [ ]:
from gensim import matutils

tfidf_dense = matutils.corpus2dense(corpus,len(dictionary)).T


In [ ]:
tfidf_dense.shape

In [ ]:
# usaremos la implementacion base de svd
from scipy.linalg import svd
from scipy.sparse.linalg import svds

u, s, vt = svds(tfidf_dense)

In [ ]:
print(u.shape)

df_tfidf = pd.DataFrame(u)

In [ ]:
print(df_reduced.shape)
print(df_tfidf.shape)
df_reduced= df_reduced.filter(["idaviso"])
df_aviso_svd = df_reduced.join(df_tfidf)
df_aviso_svd.to_csv(path_or_buf="/home/kevin/Documents/orgadatos/TP2/df_aviso_svd.csv",index= False)

# Similitud 

In [ ]:
sims = gensim.similarities.Similarity(None,tf_idf[corpus],
                                      num_features=len(dictionary))
print(sims)
print(type(sims))

In [ ]:
query_doc = [w.lower() for w in word_tokenize(postulacion_prueba)]
print(query_doc)
query_doc_bow = dictionary.doc2bow(query_doc)
print(query_doc_bow)
query_doc_tf_idf = tf_idf[query_doc_bow]
print(query_doc_tf_idf)

In [ ]:
similarity_array = sims[query_doc_tf_idf]
pos_max = 0
max_similarity = 0 
for  i in range (0,len(similarity_array)):
    if max_similarity < similarity_array[i]:
        max_similarity = similarity_array[i]
        pos_max = i
    

In [ ]:
pos_max

In [ ]:
max_similarity

In [ ]:
lista_descripciones[pos_max]

In [ ]:
len(lista_descripciones)

### K means

In [ ]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=5)


In [ ]:
# usaremos la implementacion base de svd
from scipy.linalg import svd

u, s, vt = svd(tfidf_dense)

In [ ]:
from sklearn.decomposition import TruncatedSVD
svd = TruncatedSVD(n_components=5, n_iter=7, random_state=42)
svd.fit(tfidf_dense)  

In [ ]:
from sklearn.utils.extmath import randomized_svd

U, Sigma, VT = randomized_svd(tfidf_dense, 
                              n_components=15,
                              n_iter=5,
                              random_state=None)

In [ ]:
U.shape

In [ ]:
df_tfidf = pd.DataFrame(U)

df_reduced= df_reduced.filter(["idaviso"])
df_aviso_svd = df_reduced.join(df_tfidf)
df_aviso_svd.to_csv(path_or_buf="/home/kevin/Documents/orgadatos/TP2/df_aviso_svd-15dim.csv",index= False)

In [ ]:
Sigma